## Import necessary libraries

In [16]:
%reset -f

import pickle
import pandas as pd
import numpy
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

## Save environment parameters, so those won't be saved to report file

In [17]:
environment_variables = dir()

## Define model related constants

In [18]:
# Input parameters
INPUT_TRAINING_FILE = "../data/preprocessed/difference_preprocessed_training_dataset.csv"
INPUT_EVALUATION_FILE = "../data/preprocessed/difference_preprocessed_evaluation_dataset.csv"

# Output parameters
METHOD_NAME = "naive_bayes"
TIMESTAMP = time.strftime("%d_%m_%Y-%H_%M_%S")
OUTPUT_MODEL = f"../data/models/{METHOD_NAME}_model_{TIMESTAMP}.pkl"
OUTPUT_RESULTS = f"../data/results/{METHOD_NAME}_model_{TIMESTAMP}.txt"

# Hyper parameter alternatives
HYPER_PARAMETER_VAR_SMOOTHING = list(numpy.arange(1e-9, 1, 0.01))

# Hyper parameter optimization parameters
HYPER_PARAMETER_OPTIMIZATION_SCORING = "accuracy"
HYPER_PARAMETER_OPTIMIZATION_CV = 10

# Other constants
LABELS = ["Operational", "Bankrupt"]
RANDOM_SEED = 42
VERBOSITY = 2

## Load the datasets
These dataset should contain "Bankrupt?" label and features that should be considered during learning.

In [19]:
training_dataset = pd.read_csv(INPUT_TRAINING_FILE, engine="python", delimiter=",")
training_features = training_dataset.loc[:, training_dataset.columns != "Bankrupt?"]
training_targets = training_dataset["Bankrupt?"]

evaluation_dataset = pd.read_csv(INPUT_EVALUATION_FILE, engine="python", delimiter=",")
evaluation_features = evaluation_dataset.loc[:, evaluation_dataset.columns != "Bankrupt?"]
evaluation_targets = evaluation_dataset["Bankrupt?"]

## Define the training pipeline

In [20]:
# Let's calculate prior propabilities
all_samples_count = len(training_targets.index)
bankrupt_samples_count = sum(training_targets)
operational_samples_count = all_samples_count - bankrupt_samples_count
bankrupt_prior_propability = bankrupt_samples_count / all_samples_count
operational_prior_propability = operational_samples_count / all_samples_count
prior_propabilities = [operational_prior_propability, bankrupt_prior_propability]
print(prior_propabilities)

# Create internal pipeline
classifier = GaussianNB(priors=prior_propabilities)
pipeline = Pipeline(steps=[("classification", classifier)])

# Specify the tunable hyper parameters
parameters = {
    "classification__var_smoothing": HYPER_PARAMETER_VAR_SMOOTHING,
}

# Define KFold parameters
cv = StratifiedKFold(n_splits=HYPER_PARAMETER_OPTIMIZATION_CV, shuffle=True, random_state=42)

estimator = GridSearchCV(pipeline, parameters, verbose=VERBOSITY,
    scoring=HYPER_PARAMETER_OPTIMIZATION_SCORING, cv=cv, n_jobs=-1)

[0.9677360219981668, 0.03226397800183318]


## Do training

In [21]:
estimator.fit(training_features, training_targets)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ..........classification__var_smoothing=0.010000001; total time=   0.0s
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ..........classification__var_smoothing=0.010000001; total time=   0.0s
[CV] END ..........classification__var_smoothing=0.010000001; total time=   0.0s
[CV] END ..........classification__var_smoothing=0.010000001; total time=   0.0s
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ................classification__var_smoothing=1e-09; total time=   0.0s
[CV] END ..........classification__var_smoot

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('classification',
                                        GaussianNB(priors=[0.9677360219981668,
                                                           0.03226397800183318]))]),
             n_jobs=-1,
             param_grid={'classification__var_smoothing': [1e-09, 0.010000001,
                                                           0.020000001,
                                                           0.030000001,
                                                           0.040000001,
                                                           0.050000001,
                                                           0.060000001,
                                                           0.070000001,
                                                           0.080000001,
                                                           0.090000001,
                      

## Calculate metric values

In [22]:
def confusion_matrix_to_string(cm):
    return f"True operational={cm[0][0]}, True bankrupt={cm[1][1]}, False operational={cm[0][1]}, False bankrupt={cm[1][0]}"

# Use all training data to calculate confusion matrix for training data
training_estimates = estimator.predict(training_features)
training_accuracy = balanced_accuracy_score(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix_to_string(training_confusion_matrix)

# Use model to estimate manually labeled evaluation Tweets
evaluation_estimates = estimator.predict(evaluation_features)
evaluation_accuracy = balanced_accuracy_score(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix_to_string(evaluation_confusion_matrix)

## Save trained model for future reference

In [23]:
with open(OUTPUT_MODEL, "wb") as handle:
    pickle.dump(estimator, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Save metadata and calculated statistics
This is done by saving every textual and numerical variable to file.

In [24]:
current_variables = dir()

results_string = ""

for variable in current_variables:
    # Skip environment variables and their container variable
    # Ignore also underscore variables
    if variable in environment_variables or variable == "environment_variables" or variable.startswith("_"):
        continue

    # Get variables value
    variable_value = globals()[variable]

    # If variable is numerical or string, append it to results
    if type(variable_value) is str or type(variable_value) is int or \
        type(variable_value) is float or type(variable_value) is list or \
        type(variable_value) is numpy.float64 or type(variable_value) is dict:
        results_string += f"{variable}: {variable_value}\n"

# Print results to screen
print(results_string)

# Save results to file
with open(OUTPUT_RESULTS, "w") as file:
    file.write(results_string)

HYPER_PARAMETER_OPTIMIZATION_CV: 10
HYPER_PARAMETER_OPTIMIZATION_SCORING: accuracy
HYPER_PARAMETER_VAR_SMOOTHING: [1e-09, 0.010000001, 0.020000001, 0.030000001, 0.040000001, 0.050000001, 0.060000001, 0.070000001, 0.080000001, 0.090000001, 0.100000001, 0.110000001, 0.120000001, 0.130000001, 0.140000001, 0.150000001, 0.160000001, 0.170000001, 0.180000001, 0.190000001, 0.200000001, 0.210000001, 0.220000001, 0.230000001, 0.240000001, 0.250000001, 0.26000000100000004, 0.27000000100000004, 0.28000000100000005, 0.290000001, 0.300000001, 0.310000001, 0.32000000100000003, 0.33000000100000004, 0.34000000100000005, 0.35000000100000006, 0.360000001, 0.370000001, 0.38000000100000003, 0.39000000100000004, 0.40000000100000005, 0.41000000100000006, 0.420000001, 0.430000001, 0.44000000100000003, 0.45000000100000004, 0.46000000100000005, 0.47000000100000006, 0.480000001, 0.490000001, 0.500000001, 0.510000001, 0.520000001, 0.530000001, 0.540000001, 0.550000001, 0.560000001, 0.570000001, 0.580000000999999